In [15]:
import pandas as pd 
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
df_preds = pd.read_csv(r"C:\Users\nicol\OneDrive\Master\Goldsmiths\Thesis\weather-prediction\data\df_with_all_preds.csv")

In [11]:
df_preds

,Unnamed: 0,temp,dew,humidity,precip,precipprob,precipcover,snow,snowdepth,windgust,...,preds,preds_gs,sktime_naive,preds_autoarima,preds_sarimax,sarimax_gs,lstm_base,lstm_two_layers,lstm_two_layers_batch_size_1,lstm_features
0,2023-05-22,15.1,6.9,64.3,0.000,0.0,0.00,0.0,0.0,26.1,...,14.492051,13.432552,14.401869,13.984044,15.701231,1.085501,15.214462,15.882883,14.360851,8.956800
1,2023-05-23,14.0,6.1,60.6,0.000,0.0,0.00,0.0,0.0,36.4,...,17.870735,14.786562,14.403738,13.895126,15.439488,1.225803,15.778243,15.878738,14.153891,8.830262
2,2023-05-24,15.8,6.7,61.2,0.000,0.0,0.00,0.0,0.0,34.7,...,16.441523,15.185232,14.405607,13.759057,16.274314,4.254934,16.200759,15.891689,13.845723,9.734104
3,2023-05-25,15.0,6.7,57.9,0.000,0.0,0.00,0.0,0.0,41.5,...,14.017682,13.373943,14.407477,13.725513,16.052129,3.296934,16.509863,15.915347,13.485457,8.973236
4,2023-05-26,14.2,6.9,63.5,0.000,0.0,0.00,0.0,0.0,26.1,...,13.380795,14.683416,14.409346,13.632718,14.740708,0.700043,16.757237,15.946935,13.097269,8.833356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2024-05-16,14.2,11.0,73.4,0.000,0.0,0.00,0.0,0.0,35.2,...,15.429116,16.348656,15.074766,13.488740,60.305122,6.259033,32.558770,19.365435,2.675483,10.043378
361,2024-05-17,16.4,11.7,77.8,3.598,100.0,33.33,0.0,0.0,30.5,...,16.688347,16.409649,15.076636,13.489218,60.777181,9.074756,32.558770,19.369402,2.675483,9.104539
362,2024-05-18,15.8,10.5,69.7,0.797,100.0,8.33,0.0,0.0,25.0,...,13.906611,15.812546,15.078505,13.488746,60.639595,10.413640,32.558770,19.372453,2.675483,8.920741
363,2024-05-19,16.6,12.4,89.1,10.126,100.0,20.83,0.0,0.0,18.4,...,16.043220,16.279257,15.080374,13.489213,60.068235,11.654069,32.558770,19.374568,2.675483,8.874175


In [12]:
df_preds.columns

Index(['Unnamed: 0', 'temp', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'severerisk', 'moonphase',
       'clusters_description', 'year', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'preds', 'preds_gs', 'sktime_naive',
       'preds_autoarima', 'preds_sarimax', 'sarimax_gs', 'lstm_base',
       'lstm_two_layers', 'lstm_two_layers_batch_size_1', 'lstm_features'],
      dtype='object')

In [13]:
columns_todrop = ['Unnamed: 0','dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'severerisk', 'moonphase',
       'clusters_description', 'year', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12']
df_preds = df_preds.drop(columns_todrop, axis=1)
df_preds

,temp,preds,preds_gs,sktime_naive,preds_autoarima,preds_sarimax,sarimax_gs,lstm_base,lstm_two_layers,lstm_two_layers_batch_size_1,lstm_features
0,15.1,14.492051,13.432552,14.401869,13.984044,15.701231,1.085501,15.214462,15.882883,14.360851,8.956800
1,14.0,17.870735,14.786562,14.403738,13.895126,15.439488,1.225803,15.778243,15.878738,14.153891,8.830262
2,15.8,16.441523,15.185232,14.405607,13.759057,16.274314,4.254934,16.200759,15.891689,13.845723,9.734104
3,15.0,14.017682,13.373943,14.407477,13.725513,16.052129,3.296934,16.509863,15.915347,13.485457,8.973236
4,14.2,13.380795,14.683416,14.409346,13.632718,14.740708,0.700043,16.757237,15.946935,13.097269,8.833356
...,...,...,...,...,...,...,...,...,...,...,...
360,14.2,15.429116,16.348656,15.074766,13.488740,60.305122,6.259033,32.558770,19.365435,2.675483,10.043378
361,16.4,16.688347,16.409649,15.076636,13.489218,60.777181,9.074756,32.558770,19.369402,2.675483,9.104539
362,15.8,13.906611,15.812546,15.078505,13.488746,60.639595,10.413640,32.558770,19.372453,2.675483,8.920741
363,16.6,16.043220,16.279257,15.080374,13.489213,60.068235,11.654069,32.558770,19.374568,2.675483,8.874175


In [33]:
def calculate_metrics(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(actual, predicted)
    mape = np.mean(np.abs((actual - predicted) / np.where(actual != 0, actual, np.nan))) * 100
    return mae, mse, rmse, r2, mape

In [17]:
metrics_df = pd.DataFrame(columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2', 'MAPE'])

In [31]:
metrics_list = []

# Calculate metrics for each model
for col in df_preds.columns:
    if col != 'temp':
        mae, mse, rmse, r2, mape = calculate_metrics(df_preds['temp'], df_preds[col])
        metrics_list.append({
            'Model': col,
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'R2': r2,
            'MAPE': mape
        })


metrics_df = pd.DataFrame(metrics_list)

In [32]:
metrics_df

,Model,MAE,MSE,RMSE,R2,MAPE
0,preds,2.386807,8.827414,2.971096,0.679731,83.253159
1,preds_gs,2.103032,7.130397,2.670280,0.741301,83.253159
2,sktime_naive,4.736848,31.844905,5.643129,-0.155370,83.253159
3,preds_autoarima,4.440994,27.790480,5.271668,-0.008271,83.253159
4,preds_sarimax,28.006129,1045.717374,32.337554,-36.939833,83.253159
5,sarimax_gs,10.032825,126.466961,11.245753,-3.588367,83.253159
6,lstm_base,16.584724,347.077914,18.630027,-11.592387,83.253159
7,lstm_two_layers,6.485225,62.840098,7.927175,-1.279911,83.253159
8,lstm_two_layers_batch_size_1,9.983612,126.985107,11.268767,-3.607166,83.253159
9,lstm_features,5.226126,42.029569,6.483022,-0.524881,83.253159
